In [33]:
from gensim.models import word2vec
import requests
from bs4 import BeautifulSoup
import re
import itertools
import pandas as pd, numpy as np
import os
import glob
import pathlib
import re
import janome
import jaconv
from janome.tokenizer import Tokenizer
from janome.analyzer import Analyzer
from janome.charfilter import *
import string
import MeCab
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import unicodedata
from gensim import corpora
from gensim import models
from pprint import pprint

In [13]:
ls ../../../data/wiki/wiki_data/

wiki.model                         wiki_neologd.model.trainables.syn1neg.npy
wiki.model.trainables.syn1neg.npy  wiki_neologd.model.wv.vectors.npy
wiki.model.wv.vectors.npy          wiki_wakati.txt
wiki.txt                           wiki_wakati_neologd.txt
wiki_neologd.model


In [18]:
model_neo = word2vec.Word2Vec.load("../../../data/wiki/wiki_data/wiki_neologd.model")

In [31]:
model_neo.most_similar(positive=['錦織圭'])

<ipython-input-31-2ac1ece15124>:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  model_neo.most_similar(positive=['錦織圭'])


[('シュットラー', 0.7524257898330688),
 ('クエルテン', 0.7282664775848389),
 ('デメンチェワ', 0.724471390247345),
 ('プレミア12', 0.7231545448303223),
 ('コロンビア戦', 0.7206202149391174),
 ('ウインブルドン', 0.7199490070343018),
 ('スケートアメリカ', 0.7159873247146606),
 ('スベトラーナ・クズネツォワ', 0.7125442028045654),
 ('マスターズカップ', 0.7109047770500183),
 ('竜星戦', 0.7071831822395325)]

In [23]:
ls models

model.bin


In [24]:
model = word2vec.Word2Vec.load("models/model.bin")

In [30]:
model.most_similar(positive=['錦織'])

<ipython-input-30-34c2a998ee84>:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  model.most_similar(positive=['錦織'])


[('一清', 0.8772557377815247),
 ('楠田', 0.8697320222854614),
 ('倉本', 0.8696631193161011),
 ('道明', 0.8691751956939697),
 ('美佐子', 0.8638629913330078),
 ('宏治', 0.8633940815925598),
 ('ペ・ヨンジュン', 0.860668420791626),
 ('夏木', 0.8598692417144775),
 ('裕志', 0.8572939038276672),
 ('常田', 0.855694591999054)]

In [34]:
df = pd.read_csv('/work/data/BunshuOnline/yahoo_news.csv', index_col=0)

In [35]:
df.head()

,title,category,text
0,内村航平が新型コロナ陽性,スポーツ,国際体操連盟FIGの渡辺守成会長らが29日オンラインで緊急会見を開き体操男子の種目別鉄棒で...
1,くそたれ地震 不満のみ込む今,地域,死者44人建物被害が5万棟に上った北海道胆振東部地震から2年がたった被害の大半を占める厚真...
2,GoToイート紙切れ 担当者憤り,地域,新型コロナウイルスの影響で打撃を受けた飲食業界の支援策Go To イート事業のプレミアム付...
3,週末 東日本で冷え込み強まる,国内,明日30日金は上空の寒気が日本列島の上空に南下してきます明日夜からは特に関東や東海など東日本...
4,SB周東が12戦連続盗塁 プロ新,スポーツ,ソフトバンクロッテ29日ペイペイドーム ソフトバンク周東が12試合連続盗塁のプロ野球新記録...


In [36]:
mecab = MeCab.Tagger('-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd/')
wakati = MeCab.Tagger("-Owakati -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd/")

In [37]:
def MecabMorphologicalAnalysis(path='./text.txt', output_file='wakati.txt', mecab=mecab, file=False):
    mecab_text = ''
    if file:
        with open(path) as f:
            for line in f:
                mecab_text += mecab.parse(line)
        with open(output_file, 'w') as f:
            print(mecab_text, file=f)
    else:
        for path in path.split('\n'):
            mecab_text += mecab.parse(path)
        return mecab_text

In [38]:
texts = []
for i in range(len(df)):
    texts.append(MecabMorphologicalAnalysis(df['text'][i], mecab=wakati))

In [39]:
for i, text in enumerate(texts):
    if i < 5:
        display(text[:100])

'国際体操連盟 FIG の 渡辺 守成 会長 ら が 29日 オンライン で 緊急会見 を 開き 体操 男子 の 種目 別 鉄棒 で 来夏 の 東京五輪 を 目指し て いる 内村航平 リンガーハット '

'死者 44人 建物 被害 が 5 万 棟 に 上っ た 北海道胆振東部地震 から 2年 が たった 被害 の 大半 を 占める 厚真 むか わ 安平 の 被災 3 町 の 仮設住宅 は 多い とき で'

'新型コロナウイルス の 影響 で 打撃 を 受け た 飲食 業界 の 支援策 Go To イート 事業 の プレミアム 付き 食事 券 を 巡り 京都 GoTo イート 事務局 は 28日 京都府内 '

'明日 30日 金 は 上空 の 寒気 が 日本列島 の 上空 に 南下 し て き ます 明日 夜 から は 特に 関東 や 東海 など 東日本 を 中心 に 冷え込み が 強まる 見込み です 31'

'ソフトバンクロッテ 29日 ペイペイドーム ソフトバンク 周東 が 12試合 連続 盗塁 の プロ野球 新記録 を 樹立 し た 初回 に 今季 48 個 目 の 盗塁 と なる 二 盗 に 成功 1'

In [40]:
# リストに変換
sentences = []
for text in texts:
    text_list = text.split(' ')
    sentences.append(text_list)

In [41]:
# まずdocumentsを作り、モデルに渡す。
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(sentences)]
model = Doc2Vec(documents, vector_size=100, window=5, min_count=1)

In [42]:
for i, doc in enumerate(documents):
#     if i < 10:
        print(doc[1], df['title'][i], df['category'][i], doc[0][:4])

[0] 内村航平が新型コロナ陽性 スポーツ ['国際体操連盟', 'FIG', 'の', '渡辺']
[1] くそたれ地震 不満のみ込む今 地域 ['死者', '44人', '建物', '被害']
[2] GoToイート紙切れ 担当者憤り 地域 ['新型コロナウイルス', 'の', '影響', 'で']
[3] 週末 東日本で冷え込み強まる 国内 ['明日', '30日', '金', 'は']
[4] SB周東が12戦連続盗塁 プロ新 スポーツ ['ソフトバンクロッテ', '29日', 'ペイペイドーム', 'ソフトバンク']
[5] TDR運営 511億円赤字の見通し 経済 ['オリエンタルランド', 'は', '29日', '2021年']
[6] 伊藤容疑者 事故直後の映像 国内 ['29日', 'ひき逃げ', 'など', 'の']
[7] 日本初 糖質0ビールどう実現 ライフ ['糖質ゼロ', 'と', 'いえ', 'ば']
[8] 給食で100人超食中毒 鶏に菌 国内 ['東京都', 'は', '29日', '中野']
[9] ビシエド抹消 打点王は絶望的 スポーツ ['中日', 'の', 'ダヤンビシエド', '内野手']
[10] 案里被告 参院本会議を欠席 国内 ['昨年7月', 'の', '参院選', '広島選挙区']
[11] 「THE W」決勝進出10組が決定 エンタメ ['女性芸人', 'の', '中', 'から']
[12] 巨人 きょう優勝の条件は スポーツ ['巨人', 'は', 'リーグ2', '連覇']
[13] ニトリ 島忠にTOB実施を発表 経済 ['家具', '製造', '販売', '大手']
[14] 鈴木氏の出馬断念 知事は驚き 地域 ['来年', '春', 'の', '千葉県知事選']
[15] 東京都 新たに221人感染確認 国内 ['東京都', 'に', 'よる', 'と']
[16] 伊藤容疑者 とんかつDJは公開 エンタメ ['道交法違反', 'ひき逃げ', 'など', 'の']
[17] トヨタ、9月生産が過去最高 経済 ['トヨタ自動車', 'が', '29日', '発表']
[18] GoToイベント 第1弾にUSJ 経済 ['経済産業省', 'は', '29日', '新型コロナウイルス']
[19] 歓楽

In [43]:
# 参考記事： http://qiita.com/okappy/items/32a7ba7eddf8203c9fa1
class LabeledListSentence(object):
    def __init__(self, words_list, labels):
        self.words_list = words_list
        self.labels = labels

    def __iter__(self):
        for i, words in enumerate(self.words_list):
            yield models.doc2vec.LabeledSentence(words, ['%s' % self.labels[i]])

In [44]:
companies = list(df['title'])
docs = sentences

In [45]:
companies[:5]

['内村航平が新型コロナ陽性',
 'くそたれ地震 不満のみ込む今',
 'GoToイート紙切れ 担当者憤り',
 '週末 東日本で冷え込み強まる',
 'SB周東が12戦連続盗塁 プロ新']

In [46]:
print(docs[12:13])

[['巨人', 'は', 'リーグ2', '連覇', 'へ', '優勝', 'マジック', 'を', '2', 'と', 'し', 'て', 'いる', '29日', 'に', '巨人', '勝利', '2位', 'の', '中日', 'が', '負け', 'か', '引き分け', 'の', '場合', '原監督', '率いる', '巨人', 'の', '2年連続', 'リーグ優勝', 'が', '決まる', '巨人', 'が', '引き分け', 'た', '場合', 'は', '中日', 'が', '勝っ', 'て', 'も', '負け', 'て', 'も', '引き分け', 'て', 'も', 'マジック1', 'に', 'なる', '巨人', 'は', '横浜スタジアム', 'で', 'の', 'DeNA', '戦', 'で', '予告先発', 'は', 'サンチェス', 'と', '平良', '中日', 'は', '甲子園', 'で', 'の', '阪神戦', 'で', '予告先発', 'は', '大野', '雄', 'と', '西', '勇', '両', '球場', 'とも', '午後6時', 'の', '同', '時刻', 'に', '開始', 'と', 'なる', '\n']]


In [47]:
texts = LabeledListSentence(docs, companies)

In [57]:
model = models.Doc2Vec(alpha=0.025, min_count=1,
                       size=100, iter=20, workers=4)

/opt/anaconda3/lib/python3.8/site-packages/gensim/models/doc2vec.py:315: UserWarning: The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.
  warnings.warn("The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.")
/opt/anaconda3/lib/python3.8/site-packages/gensim/models/doc2vec.py:319: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


In [58]:
model.build_vocab(texts)

<ipython-input-43-20ff57f010e2>:9: DeprecationWarning: Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).
  yield models.doc2vec.LabeledSentence(words, ['%s' % self.labels[i]])


In [61]:
model.intersect_word2vec_format('/work/data/wiki/wiki_data/wiki_wakati_neologd.txt', binary=False)

AttributeError: 'Doc2Vec' object has no attribute 'intersect_word2vec_format'

In [55]:
ls /work/data/wiki/wiki_data/

wiki.model                         wiki_neologd.model.trainables.syn1neg.npy
wiki.model.trainables.syn1neg.npy  wiki_neologd.model.wv.vectors.npy
wiki.model.wv.vectors.npy          wiki_wakati.txt
wiki.txt                           wiki_wakati_neologd.txt
wiki_neologd.model


In [63]:
model_neo_copy = model_neo

In [67]:
model_neo_copy.train(texts, total_words=17088, epochs=1)

<ipython-input-43-20ff57f010e2>:9: DeprecationWarning: Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).
  yield models.doc2vec.LabeledSentence(words, ['%s' % self.labels[i]])
Exception in thread Thread-25:
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/opt/anaconda3/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/anaconda3/lib/python3.8/site-packages/gensim/models/base_any2vec.py", line 210, in _worker_loop
    tally, raw_tally = self._do_train_job(data_iterable, job_parameters, thread_private_mem)
  File "/opt/anaconda3/lib/python3.8/site-packages/gensim/models/word2vec.py", line 638, in _do_train_job
    tally += train_batch_cbow(self, sentences, alpha, work, neu1, self.compute_loss)
  File "gensim/models/word2vec_inner.pyx", line 631, in gensim.models.word2vec_inner.train_batch_cbow
Ty

KeyboardInterrupt: 

In [65]:
len(model.wv.vocab)

17088